In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML

# =============================
# UI Elements
# =============================
start_input = widgets.IntText(
    description="Start from N:",
    value=10,
    style={'description_width': 'initial'}
)

mode_dropdown = widgets.Dropdown(
    options=[
        ('Include N if prime (≥ N)', 'inclusive'),
        ('Strictly greater than N (> N)', 'exclusive')
    ],
    value='inclusive',
    description='Mode:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

start_button = widgets.Button(
    description="Start",
    button_style='primary',
    tooltip="Initialize the generator from N"
)

next_button = widgets.Button(
    description="Next prime ▶",
    tooltip="Find the next prime",
)

reset_button = widgets.Button(
    description="Reset",
    tooltip="Clear results and start over",
)

quit_button = widgets.Button(
    description="Quit",
)

html_output = widgets.HTML()
error_output = widgets.HTML()

controls_row1 = widgets.HBox([mode_dropdown, start_button, next_button, reset_button])
display(start_input, controls_row1, error_output, html_output, quit_button)


# =============================
# Prime utilities
# =============================
def _decompose_pow2(n: int):
    """Write n as d * 2^s with d odd. Return (s, d)."""
    s = 0
    d = n
    while d % 2 == 0:
        s += 1
        d //= 2
    return s, d

def _miller_rabin(n: int) -> bool:
    """
    Deterministic Miller–Rabin for 64-bit integers using a fixed base set.
    For larger ints, it's a strong probable-prime test.
    """
    if n < 2:
        return False
    # Small primes quick check
    small_primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37]
    if n in small_primes:
        return True
    for p in small_primes:
        if n % p == 0:
            return False

    # Write n-1 = d * 2^s
    s, d = _decompose_pow2(n - 1)

    # Deterministic for 64-bit using these bases
    # (A superset of necessary bases; still fast in practice)
    bases = [2, 3, 5, 7, 11, 13, 17]

    def trial(a):
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            return True
        for _ in range(s - 1):
            x = (x * x) % n
            if x == n - 1:
                return True
        return False

    for a in bases:
        if a % n == 0:
            continue
        if not trial(a):
            return False
    return True

def is_prime(n: int) -> bool:
    """Return True if n is prime."""
    return _miller_rabin(n)

def next_prime_from(k: int) -> int:
    """Return the smallest prime ≥ k (if k <= 2 returns 2)."""
    if k <= 2:
        return 2
    # Ensure odd candidate
    candidate = k if k % 2 == 1 else k + 1
    while not is_prime(candidate):
        candidate += 2  # skip even numbers
    return candidate


# =============================
# State
# =============================
_state = {
    "initialized": False,
    "start": 10,
    "mode": "inclusive",
    "primes": [],
    "last_prime": None
}


# =============================
# Rendering
# =============================
def _chips_html(items):
    colors = [
        "#FF6B6B", "#4ECDC4", "#45B7D1", "#F9A602",
        "#9B59B6", "#E74C3C", "#3498DB", "#2ECC71",
        "#F1C40F", "#1ABC9C"
    ]
    chip_html = []
    for i, val in enumerate(items):
        color = colors[(i // 10) % len(colors)]
        chip_html.append(
            f"""
            <span style="
                border: 2px solid {color};
                color: {color};
                font-style: italic;
                padding: 4px 8px;
                margin: 2px 4px;
                font-family: monospace;
                display: inline-block;
                border-radius: 8px;
            ">{val}</span>
            """
        )
    return ''.join(chip_html) if items else '<div style="color:#888;">(none yet)</div>'

def _render():
    primes = _state["primes"]
    start = _state["start"]
    mode = _state["mode"]
    initialized = _state["initialized"]

    mode_label = "≥ N" if mode == "inclusive" else "> N"
    count = len(primes)
    last_val = primes[-1] if primes else "—"

    title = f"Next Prime Numbers from N = {start} ({mode_label})"
    body = _chips_html(primes)

    html_output.value = f"""
    <div style="font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial; max-width: 900px;">
        <div style="text-align: center; margin-bottom: 10px;">
            <h3 style="font-weight: 700; margin: 0;">{title}</h3>
            <div style="color:#444; font-size: 13px; margin-top: 6px;">
                {'Click "Start" to initialize.' if not initialized else 'Click "Next prime ▶" to continue.'}
                &nbsp; Count: <b>{count}</b> &nbsp;|&nbsp; Last prime: <b>{last_val}</b>
            </div>
        </div>
        <div style="border: 1px solid #ddd; border-radius: 10px; padding: 10px;
                    max-height: 320px; overflow: auto; background: #fafafa;">
            {body}
        </div>
        <div style="color:#888; font-size: 12px; margin-top: 8px;">
            Note: Uses a deterministic Miller–Rabin set for 64-bit integers; beyond that it's a strong probable-prime test.
        </div>
    </div>
    """


# =============================
# Event handlers
# =============================
def _initialize():
    error_output.value = ""
    try:
        n = start_input.value
        mode = mode_dropdown.value
        if n is None or not isinstance(n, int):
            error_output.value = "<div style='color:red;'>Please enter a valid integer for N.</div>"
            return

        _state["start"] = n
        _state["mode"] = mode
        _state["primes"].clear()
        _state["last_prime"] = None

        # Determine first prime based on mode
        if mode == "inclusive":
            candidate = max(2, n)
        else:  # exclusive
            candidate = max(2, n + 1)

        first = next_prime_from(candidate)
        _state["primes"].append(first)
        _state["last_prime"] = first
        _state["initialized"] = True

        _render()

    except Exception as err:
        error_output.value = f"<div style='color:red;'>An error occurred: {err}</div>"

def on_start_clicked(b):
    _initialize()

def on_next_clicked(b):
    error_output.value = ""
    try:
        if not _state["initialized"]:
            error_output.value = "<div style='color:#c0392b;'>Click <b>Start</b> first to initialize.</div>"
            return

        last_prime = _state["last_prime"]
        candidate = last_prime + 1
        nxt = next_prime_from(candidate)

        _state["primes"].append(nxt)
        _state["last_prime"] = nxt

        _render()

    except Exception as err:
        error_output.value = f"<div style='color:red;'>An error occurred: {err}</div>"

def on_reset_clicked(b):
    error_output.value = ""
    _state["initialized"] = False
    _state["primes"].clear()
    _state["last_prime"] = None
    _render()

def on_quit_button_clicked(b):
    # Clear output and close the widgets
    html_output.value = ""
    error_output.value = ""
    start_input.close()
    mode_dropdown.close()
    start_button.close()
    next_button.close()
    reset_button.close()
    quit_button.close()
    html_output.close()
    error_output.close()

start_button.on_click(on_start_clicked)
next_button.on_click(on_next_clicked)
reset_button.on_click(on_reset_clicked)
quit_button.on_click(on_quit_button_clicked)

# Initial render
_render()

IntText(value=10, description='Start from N:', style=DescriptionStyle(description_width='initial'))

HTML(value='')

HTML(value='')

Button(description='Quit', style=ButtonStyle())